In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from IPython.display import clear_output

In [3]:
!pip install gdown
!pip install transformers
!pip install sentencepiece
clear_output()

In [4]:
!gdown https://drive.google.com/uc?id=1M4KnsDq_xeqACZQi30NLT1CYzqHl6b21
!gdown https://drive.google.com/uc?id=1fGx-JTtk41GMqMvxbGfksLE73Scbx_IV

Downloading...
From: https://drive.google.com/uc?id=1M4KnsDq_xeqACZQi30NLT1CYzqHl6b21
To: /content/streets_cheb.csv
100% 27.2k/27.2k [00:00<00:00, 47.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=1fGx-JTtk41GMqMvxbGfksLE73Scbx_IV
To: /content/cheb_streets_houses.csv
100% 389k/389k [00:00<00:00, 52.1MB/s]


In [5]:
!wget https://clck.ru/YT75w -O data.zip
!unzip -u data.zip

--2021-10-30 07:10:22--  https://clck.ru/YT75w
Resolving clck.ru (clck.ru)... 213.180.204.221, 2a02:6b8::221
Connecting to clck.ru (clck.ru)|213.180.204.221|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://sba.yandex.net/redirect?url=https%3A%2F%2Fstorage.yandexcloud.net%2Fhackai-datasetrsv%2FArchive.zip&client=clck&sign=b351c9bcc3485cf0572cef79be48d6df [following]
--2021-10-30 07:10:23--  https://sba.yandex.net/redirect?url=https%3A%2F%2Fstorage.yandexcloud.net%2Fhackai-datasetrsv%2FArchive.zip&client=clck&sign=b351c9bcc3485cf0572cef79be48d6df
Resolving sba.yandex.net (sba.yandex.net)... 87.250.250.232, 93.158.134.232, 213.180.204.232, ...
Connecting to sba.yandex.net (sba.yandex.net)|87.250.250.232|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://storage.yandexcloud.net/hackai-datasetrsv/Archive.zip [following]
--2021-10-30 07:10:23--  https://storage.yandexcloud.net/hackai-datasetrsv/Archive.zip
Resolving st

In [6]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from nltk.stem.snowball import SnowballStemmer
from tqdm.notebook import tqdm
from functools import lru_cache
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

stemmer = SnowballStemmer("russian") 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [7]:
@lru_cache(maxsize=None)
def stem(text):
    return stemmer.stem(text)

In [8]:
model_checkpoint = 'cointegrated/rubert-base-cased-nli-twoway'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)
if torch.cuda.is_available():
    model.cuda()

Downloading:   0%|          | 0.00/545 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.57M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.49M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/679M [00:00<?, ?B/s]

In [51]:
df = pd.read_csv("/content/cheb_streets_houses.csv")
df["address"] = df["street"].apply(lambda x: " ".join([i.strip() for i in x.split(",")[::-1]])) + " " + df["house"]

In [38]:
df.head(4)

,street,house,address
0,"139-й Стрелковой Дивизии, ул.",3,ул. 139-й Стрелковой Дивизии 3
1,"139-й Стрелковой Дивизии, ул.",4,ул. 139-й Стрелковой Дивизии 4
2,"139-й Стрелковой Дивизии, ул.",5,ул. 139-й Стрелковой Дивизии 5
3,"139-й Стрелковой Дивизии, ул.",6,ул. 139-й Стрелковой Дивизии 6


In [39]:
encode = {"ул.": "Улица ул.",
          "пр-т": "Проспект пр-т",
          "пр-д": "Проезд пр-д",
          "пер.": "Перекресток пер.",
          "б-р": "Бульвар б-р",
          "ш.": "Шоссе ш.",
          "пл.": "Площадь пл.",
          "наб.": "Набережная наб."}

In [52]:
df["address"].value_counts()

ул. Базарная 12              2
пр-д Лапсарский 13           2
ул. Плеханова 16             1
ул. Плеханова 37             1
ул. Энергетиков 18           1
                            ..
ул. Прибрежная 4             1
ул. Гражданская 61           1
ул. 40 лет Октября 3-я 13    1
ул. Кольцевая 25             1
ул. Дегтярёва 42В            1
Name: address, Length: 10854, dtype: int64

In [43]:
# for key, value in encode.items():
#     df["address"] = df["address"].str.replace(key, value)

In [53]:
df.head(5)

,street,house,address
0,"139-й Стрелковой Дивизии, ул.",3,ул. 139-й Стрелковой Дивизии 3
1,"139-й Стрелковой Дивизии, ул.",4,ул. 139-й Стрелковой Дивизии 4
2,"139-й Стрелковой Дивизии, ул.",5,ул. 139-й Стрелковой Дивизии 5
3,"139-й Стрелковой Дивизии, ул.",6,ул. 139-й Стрелковой Дивизии 6
4,"139-й Стрелковой Дивизии, ул.",7,ул. 139-й Стрелковой Дивизии 7


In [45]:
df["address"].unique()

array(['Улица ул. 139-й Стрелковой Дивизии 3',
       'Улица ул. 139-й Стрелковой Дивизии 4',
       'Улица ул. 139-й Стрелковой Дивизии 5', ...,
       'Улица ул. Ярославская 76', 'Улица ул. Ясеневая 5',
       'Улица ул. Ясеневая 19'], dtype=object)

In [54]:
classes = df["address"].to_list()
n_classes = len(classes)

In [60]:
def predict_zero_shot(text, label_texts, model, tokenizer,
                      label='entailment', normalize=True):
    text = " ".join([stem(i) for i in word_tokenize(text)])
    tokens = tokenizer([text] * n_classes, label_texts,
                       truncation=True, return_tensors='pt', padding=True)
    with torch.inference_mode():
        result = torch.softmax(model(**tokens.to(model.device)).logits, -1)
    proba = result[:, model.config.label2id[label]].cpu().numpy()
    if normalize:
        proba /= sum(proba)
    return label_texts[proba.argmax()]
    return sorted(zip(label_texts, proba), reverse=True, key=lambda x: x[1])[:10]

In [61]:
predict_zero_shot('город Чебоксары улица  Лебедева 9', classes, model, tokenizer)

'ул. Лебедева 9'

In [ ]:
df2 = pd.read_excel("/content/обращения граждан на портале народный контроль.xlsx").dropna(subset=["Текст обращения"])

In [ ]:
arr = []
for i in tqdm(df2["Текст обращения"]):
    arr.append(predict_zero_shot(i, classes, model, tokenizer))

  0%|          | 0/3137 [00:00<?, ?it/s]

RuntimeError: ignored

In [ ]:
df2["street"] = arr

In [ ]:
df2.to_csv("/content/drive/MyDrive/обращения_граждан_clear.csv", index=False)